In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from WindPy import w
w.start()
from function import *
from nav_research import NavResearch
from pathlib import Path
import os
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter('ignore')

plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2024 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


In [14]:
folder_path = Path("docs")

In [15]:
html_files = list(folder_path.rglob("*.html"))

In [21]:
html_files[1].parent

WindowsPath('docs/主观CTA')

In [17]:
unique_folders = {html_file.parent for html_file in html_files if html_file.name != "index.html"}
unique_folders

{WindowsPath('docs/主观CTA'), WindowsPath('docs/套利'), WindowsPath('docs/量化CTA')}

In [ ]:
from pathlib import Path

def generate_index_html(folder_path: Path):
    # 获取目录及其子目录中的所有 HTML 文件
    html_files = list(folder_path.rglob("*.html"))
    
    # 创建一个集合来存储所有唯一的文件夹路径
    unique_folders = {html_file.parent for html_file in html_files if html_file.name != "index.html"}
    
    # 对文件夹进行排序（这里使用自然排序，但可以根据需要更改）
    sorted_folders = ["主观CTA","量化CTA","套利"]
    
    # 创建一个新的 index.html 文件
    with open(folder_path.joinpath("index.html"), "w", encoding="utf-8") as f:
        f.write(
            """<html>
<head>
<meta charset="UTF-8">
<title>Value over Time</title>
<style>
    body {
        font-family: Arial, sans-serif;
        font-size: 20px;
        color: blue;
    }
    h2 {
        font-size: 25px;
        color: blue;
    }
    a {
        text-decoration: none;
    }
</style>
</head>
<body>"""
        )
        
        # 为每个文件夹创建一个标题，并列出其下的所有HTML文件
        for folder in sorted_folders:
            folder_name = folder.name
            f.write(f"<h2><a href='{folder.relative_to(folder_path)}'>{folder_name}</a></h2>\n")
            f.write("<ul>\n")
            for html_file in html_files:
                if html_file.parent == folder:
                    cleaned_name = html_file.name.replace("_nav_analysis", "")
                    relative_path = html_file.relative_to(folder_path)
                    f.write(f'  <li><a href="{relative_path}" target="_blank">{cleaned_name}</a></li>\n')
            f.write("</ul>\n")
        
        f.write("</body>\n</html>")

# 使用函数
generate_index_html(Path("docs"))